In [182]:
import pandas as pd
import re
import numpy as np

In [183]:
jobs = ['Data scientist', 'Python', 'JS', 'Java','Data scientist', 'Python', 'JS', 'Java']
countrys = ['France','France','France','France', 'USA','USA','USA','USA']
filenames = ['data/'+str.lower(job).replace(' ','')+str.lower(country)+'.csv' for job, country in zip(jobs,countrys) ]

df = pd.concat([pd.read_csv(filename, sep=';', encoding="utf-8-sig", index_col=0) for filename in filenames])

In [184]:
skills = ['Python', 'R', 'SQL', 'NoSQL', 'GIT', 'Spark', 'Flask', 'Streamlit', 'Docker', 'Kubernetes', 'React', 'Machine-Learning', 'Deep Learning', 'NLP', 'Vue', 'Angular', 'Scala', 'PySpark','PowerBI', 'SQLServer', 'Dataiku','Data', 'Keras', 'Tensor-Flow', 'NLU', 'PyTorch', 'Scikit-Learn', 'SAAS', 'Java', 'Scikit learn', 'Hadoop', 'Hive', 'ML','DL', 'Azure', 'AWS']

# contrats = ['CDI','CDD','Apprentissage','Contrat pro','Stage','Intérim','Freelance','Indépendant','Freelance']
            
studylevel_keywords = {'High School':['High School','Baccalauréat'],
                       'Guaduate': ['Université','Universitaire','Formation supérieure','Guaduate','Bac +2','Bac +3','Bachelor'],
                       'Master': ['Master','Master 2', 'Master +2', 'Bac +5', 'Grande école d\'ingénieur','Bac +4/5','M2','Cursus ingénieur', 'engineer','Diplôme ingénieur'],
                       'Doctor': ['Doctorat', 'Doctor'], 
                       'Formation': ['Formation Data Science']
                       }

explevel_keywords = {'Beginer':['Débutant', 'Beginer','No experience','Pas d`\'expérience', '1 an', '0-1 an','0 - 1 an', '0/1 an','0 / 1 an'],
                       'Intermediate': ['Intermédiaire','Intermediate','2 year', '3 year','2-3 year',  '2 ans', '2/3 an', '1-2 an','1-3  years'],
                       'Confirmed': ['Confirmé','Confirmed','3 an','4 an','5 an','3 year','4 year','5 year','3-5 year','3-5 an','3/5 year', '3/5 an'],
                       'Expert': ['Expert','6 year','7 year', '8 year', '9 year','6 an','7 an', '8 an','9 an','10 an','5-10 year','5-10 an','5 - 10 an','5/10 year', '5/10 an'],
                       'Senior': ['Senior', '10+ years', '10+ an', '10 ans +','plus de 10 an']
                       }

In [185]:
# La fonction vérifie si un terme existe dans une chaîne de caractère
def exist_in_array(x, dicto):

    for key in dicto:
        if any(str.lower(term) in str.lower(x) for term in dicto[key]):
            return key


# La fonction vérifie si dans la colonne ['Description'] de chaque ligne, s'il trouve descompétences dans le dictionnaire, si oui, il met un 1
def skill_required(row, skills):

    desc_low = str.lower(row['Description'])
    
    for skill in skills:

        skill_low = str.lower(skill)

        if (skill_low in desc_low) or (skill_low.replace(' ','') in desc_low) or (skill_low.replace('-','') in desc_low)  or (skill_low.replace('-',' ') in desc_low):
            row[skill] = 1

    return row

# La fonction retourne le résultat obtenu à partir d'un string et regex
def searchBy(x,pattern):
    match = re.search(pattern,str.lower(x))
    if match:
        return match.group(0)

# La fonction remplace valeurs nulles de la colonne ['Salaire'] par ['SalaryExtract'], si ['SalaryExtract'] n'est pas NaN
def replace_empty_salary(row):

    if row['SalaryExtract'] is not None: 
        if row['Salaire'] == 'None':
            row['Salaire'] = row['SalaryExtract']
        elif type(row['Salaire']) == float:
            row['Salaire'] = row['SalaryExtract']
        else:
            pass
    return row 


# Fonction qui retourne un numbre de format correcte à partir des nombres en format de string
def correct_number(string):
    numb_string = str(string).replace(' ','')

   # Si le nombre a un virgule devant les deux dernier chiffres 
    if re.match('^.+[.]\d{2}$',numb_string):
        numb = numb_string.replace(',','')
        return(float(numb))
        
   # Si le nombre a un point devant les deux dernier chiffres
    elif re.match('^.+[,]\d{2}$',numb_string):
        numb = numb_string.replace(',','.')
        return(float(numb))
    
   # Si le nombre n'a pas de point ni virgule devant les deux dernier chiffres
    else:
        numb = numb_string.replace(',','').replace('\.','')
        return(float(numb))


# fonction qui sépare les salaire minimum et salaire maximum en 2 colonnes différents
def minmaxSalary(row):
    match = re.match(r'(?:\D*)(?:[\$€]?\s?)((?:\d+)(?:[\.,\s]?\d{3})*(?:[\s]?k)?(?:[\.,\s]\d{2})?)(?:[\s]?[\$€]?[\s]?(?:à|au|et|\/|->?|to|and)?[\s]?[\$€]?\s?)((?:\d+)(?:[\.,\s]?\d{3})*(?:[\s]?k)?(?:[\.,\s]\d{2})?)?',row['Salaire'])

    # Si on obtient les salaire minimum et salaire maximum
    if (len(match.groups(0))==2) and (match.groups(0)[1] != 0):

        minSalary = correct_number(match.groups(0)[0])
        maxSalary = correct_number(match.groups(0)[1])

    # Si on obtient juste un salaire
    elif match.groups(0)[1] == 0:

        minSalary = correct_number(match.groups(0)[0])
        maxSalary = minSalary

    # Non retrouvé
    else:

        minSalary = np.nan
        maxSalary = np.nan

    #Si dans le salaire est indiqué le salaire par an
    if re.search(r'(annually)|(year)|(((par)|(per)|(by)|(\/)|(every)|(un)|(une))[\s]?an)',row['Salaire']):
        row['Salaire minimum par an'] = minSalary
        row['Salaire maximum par an'] = maxSalary

    #Si dans le salaire est indiqué le salaire par mois
    elif re.search(r'(month)|(mensuel)|(mois)',row['Salaire']):
        row['Salaire minimum par an'] = minSalary * 12
        row['Salaire maximum par an'] = maxSalary * 12

    #Si dans le salaire est indiqué le salaire par semaine
    elif re.search(r'(week)|(semaine)|(hebdomadaire)',row['Salaire']):
        row['Salaire minimum par an'] = minSalary *4 * 12
        row['Salaire maximum par an'] = maxSalary *4 * 12
    
    #Si dans le salaire est indiqué le salaire par jour
    elif re.search(r'(day)|(jour)',row['Salaire']):
        row['Salaire minimum par an'] = minSalary *22 * 12
        row['Salaire maximum par an'] = maxSalary *22 * 12
    
    #Si dans le salaire est indiqué le salaire par heure
    elif re.search(r'(hour)|(heur)',row['Salaire']):
        row['Salaire minimum par an'] = minSalary *8 * 22  * 12
        row['Salaire maximum par an'] = maxSalary * 8 * 22  * 12
        
    #pour tout autres cas return np.nan
    else:
        row['Salaire minimum par an'] = np.nan
        row['Salaire maximum par an'] = np.nan
    return row
         

In [186]:
df['Study required'] = df['Description'].apply(lambda x: exist_in_array(x, studylevel_keywords))
df['Experience required'] = df['Description'].apply(lambda x: exist_in_array(x, explevel_keywords))

df = pd.concat([df, pd.DataFrame(columns=skills)])
df = df.apply(lambda row: skill_required(row, skills), axis=1)

In [187]:
salaryMatch = re.compile(r'((([\$€]\s?\d+([\.,\s]?\d{3})*[\s]?k?([\.,\s]\d{2})?([\s]?((à)|(au)|(et)|(\/)|(->?)|(to)|(and))?[\s]?[\$€]?[\s]?\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]?\d{2})?)?)|((\d+([\.,\s]?\d{3})*\s?k?([\.,\s]\d{2})?\s?[\$€]?([\s]?((à)|(au)|(et)|(and)|(\/)|(->?)|(to))?[\s]?\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]\d{2})?)?)[€\$])[\s]?)(\s*(brut)?\s*((par)|(a)|(an)|(per)|(by)|(\/)|(every)|(un)|(une))[\s]*((annually)|(heure)|(jour)|(hebdomadaire)|(mois)|(mensuel)|(hour)|(day)|(month)|(year)|(an))))')

In [188]:
df['SalaryExtract'] = df['Description'].apply(lambda x: searchBy(x ,salaryMatch))
df = df.apply(lambda row: replace_empty_salary(row), axis=1)
df = df[df['Salaire']!= 'None']
df['Salaire'].dropna(inplace=True)
df.drop('Type de contrat', axis=1, inplace=True)
df.drop('SalaryExtract',axis=1, inplace=True)
df = df.drop_duplicates()
df = pd.concat([df, pd.DataFrame(columns=['Salaire minimum par an','Salaire maximum par an'])])
df = df.apply(lambda row: minmaxSalary(row), axis=1)
df = df[df['Salaire minimum par an']>10000]

In [189]:
df

,Entreprise,Pays,Ville,Salaire,Description,Study required,Experience required,Python,R,SQL,...,Java,Scikit learn,Hadoop,Hive,ML,DL,Azure,AWS,Salaire minimum par an,Salaire maximum par an
Data Scientist H/F,AGYLIS,France,La Défense (92),40 000 € - 50 000 € par an,AGYLIS est un acteur important des Entreprise ...,Master,Beginer,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,40000.0,50000.0
DATA SCIENTIST,Servier,France,Suresnes (92),65 000 € par an,VOS MISSIONS : QUI NOUS SOMMES : Servier est u...,Guaduate,Confirmed,1.0,1.0,NaN,...,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,65000.0,65000.0
Data scientist / machine learning engineer (H/F) - Expérienc...,ABSIS CONSEIL,France,Fontenay-sous-Bois (94),55 000 € - 65 000 € par an,Descriptif de la mission (au sein d'une équipe...,None,Expert,1.0,1.0,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,55000.0,65000.0
Data Scientist / Experience analyst,LIVEMENTOR,France,Paris (75),40 000 € - 50 000 € par an,À propos LiveMentor est la référence du coachi...,Master,Beginer,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,50000.0
Data Scientist Confirmé,AGYLIS,France,Paris (75),3 029 € - 3 703 € par mois,AGYLIS est un acteur important des Entreprise ...,Master,Confirmed,1.0,1.0,1.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,36348.0,44436.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sr. Software Engineer,GrayMatter Consulting Services,USA,"Alpharetta, GA 30004","$111,384 a year",Require: Bachelor's degree or a foreign equiva...,Guaduate,None,NaN,1.0,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111384.0,111384.0
"Sr. Software Engineer - Full Stack, Angular and Java",Amway Inc.,USA,"Ada, MI","$96,000 a year","Job title: Sr. Software Developer – Angular, J...",Master,Expert,NaN,1.0,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,96000.0,96000.0
Senior Software Engineering Careers,Liberty Mutual Insurance,USA,"Boston, MA 02101","$95,000 - $170,000 a year",Thank you for considering software careers at ...,Master,Expert,NaN,1.0,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,95000.0,170000.0
Senior Full Stack Developer,SCALAR,USA,"Draper, UT","$100,000 - $145,000 a year",Job Description Scalar is hiring a Senior Full...,Guaduate,Senior,1.0,1.0,1.0,...,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,100000.0,145000.0


In [179]:
df['Classe de salaire'] = np.nan
